# 浅谈Python的shallow copy和deep copy

先导入copy模块

In [1]:
import copy 

对于简单的object，用**shallow copy**和**deep copy**是没有任何区别的，首先先来看下面的一个例子

In [4]:
original = 1
copy1 = copy.copy(original) #copy1是original的shallow copy
copy2 = copy.deepcopy(original) #copy2是original的deep copy
original = 2
print(original)
print(copy1)
print(copy2)
copy1 == copy2
copy1 is copy2

2
1
1


True

通过结果发现，copy1和copy2都不会随original改变而改变

而对于复杂的object，例如list中套list的情况，shallow copy中的子list，其实并没有从object种真正的**独立**出来

通俗来讲就是，当改变原object中的子list时，shallow copy出来的值会跟着一起改变，看下面这个例子

In [5]:
original = [1,2,[3,4]]
copy1 = copy.copy(original)
copy2 = copy.deepcopy(original)
copy1 == copy2
copy1 is copy2

False

此时已经可以看到copy1和copy2虽然值相同，但是已经不再是同一个object了

然后现在来改变original的值

In [6]:
original[2][0] = 'hello!'

再来看original，copy1和copy2的值

In [7]:
print(original)
print(copy1)
print(copy2)

[1, 2, ['hello!', 4]]
[1, 2, ['hello!', 4]]
[1, 2, [3, 4]]


可以发现，copy1，也就是shallow copy的值是跟着original改变了的。而copy2，也就是deep copy的值是不会随着原始副本的值改变而改变的

至此，似乎deep copy更符合我们对**复制**的理解，被复制的东西应该是独立存在，不受原始副本影响的。如果我们想要获得我们理解的**复制**，那么就用deep copy就行了。

可是，为什么Python提供了shallow copy这种copy的方式，这就和Python的变量存储方式有关了

## Python的变量储存方法

对于Python而言，变量是对内存及其地址的抽象。Python中的一切变量都是对象。变量的存储，采用了引用语义的方式，存储的只是一个变量的值所在的内存地址，而不是这个变量的值本身。

例如在Python中，a = XX，其实可以看成是给XX贴上了一个标签a。当再次赋值给a的时候，就相当于把a这个标签从XX上取下来。这就可以解释Python中可能遇到到一些诡异情况。

In [9]:
a = [1,2,3]
b = a
a = [4,5,6]
print(a,b)

[4, 5, 6] [1, 2, 3]


可以看到，a的值改变后，b的值并不随之变化。

In [11]:
a = [1,2,3]
b = a
a[0],a[1],a[2] = 4,5,6
print(a,b)

[4, 5, 6] [4, 5, 6]


但是这里却发现a的值改变后，b的值也随之改变了

在上面的两段代码中，a的值都发生了改变。但是区别在于，在第一段代码里是直接赋给了a新的值([4,5,6])。而在第二段代码里则是把list中的每个值分别改变了。

对于上述这种情况，其实可以把[1,2,3]看成是一个物品。执行 'a = [1,2,3]'其实可以看做是给[1,2,3]这个物品贴上了a这个标签。而执行'b = a'后，可以看做是又给[1,2,3]这个物品贴上了b的标签。

## 对第一种代码的解释 

a = [4,5,6]其实相当于把[1,2,3]上的a这个标签撕下来，贴在了[4,5,6]上。

而在这个过程中，[1,2,3]这个物品其实并没有消失。标签b自始至终都贴在[1,2,3]上的，既然这个reference没有被改变，则b的值当然也不会变。

## 对第二种代码的解释 

执行 a[0],a[1],a[2] = 4,5,6 这一句后，其实已经直接改变了[1,2,3]这个物品本身，这个物品被改变之后已经变成了[4,5,6]

而在这个过程中，标签a和b都没有被移动，它们还是在[1,2,3]这个物品上，因此a和b的值就是[4,5,6]

### 对于copy的问题，官方的解释是

> The difference between shallow and deep copying is only relevant for compound objects (objects that contain other objects, like lists or class instances):

> A shallow copy constructs a new compound object and then (to the extent possible) inserts references into it to the objects found in the original.

> A deep copy constructs a new compound object and then, recursively, inserts copies into it of the objects found in the original.

**shallow copy和deep copy只有在面对复合对象时才有区别，所谓复合对象就是，是指对象中含其他对象(如list中套list或class)等**。

**由 shallow copy 建立的新复合对象中，每个子对象，都只是指向自己在原来本体中对应的子对象。而 deep copy 建立的复合对象中，存储的则是本体中子对象的 copy，并且会层层如此 copy 到底。**

这个解释略显抽象，通俗地来讲就是：

shallow copy其实是给原始副本建立了一个镜像，original 当中的元素指向哪， copy1 中的元素就也指向哪。这就是官方 doc 文档中所说的 inserts references into it to the objects found in the original 。original[2]，也就是 [3, 4] 这个 list。根据 shallow copy 的定义，在 copy1[2] 指向的就是同一个 list [3, 4]。那么，如果我们改变了这个 list，就会导致 original 和 copy1 同时改变。这就是为什么上边 original[2][0] = "hey!" 之后，copy1 也随之变成了 [1, 2, ['hey!', 4]]。

deep copy其实是把原始副本的每一层都copy了一份存储起来，这时候的 original[2] 和 copy2[2] 虽然值都等于 [3, 4]，但已经不是同一个 list了，所以就是完全独立的，那么无论怎样改变其中的值，另一个的值都不会随之改变。